# Workshop 4: Poisson and least-squares problems

We now turn our attention to least-squares problems on the gradient of the solution 

$$
\min_u \frac{1}{2} \int_{\Omega } || \nabla u(X) - F(X) ||_2^2 \partial \Omega ,
$$

and on the solution itself (function approximation by error minimization) 

$$
\min_u \frac{1}{2} \int_{\Omega } \left[ u(X) - f(X) \right]^2 \partial \Omega .
$$

## Poisson equation

1. Show that global minimizers to the gradient error minimization are solutions to the Poisson equation

$$
\Delta u(X) = \nabla \cdot F(X), \quad X \in \Omega .
$$

2. Discretize the Poisson equation using FEM (on paper). If $F(X)$ is discretized at element quadrature points into $F \in \mathbb{R}^{d|E|q \times 1}$, you should get $L u = -D [I_{d \times d} \otimes Q] F$. If $F(X)$ is discretized at the nodes into $F \in \mathbb{R}^{dn \times 1}$, you should get $L u = -D [I_{d \times d} \otimes Q N] F$.

### Heat method for geodesic distance computation

One (of many) situations in which the Poisson equation shows up is in the [Heat method](https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/) for geodesic distance computation. In a nutshell, geodesic distances are lengths of the shortest paths between 2 points, where all possible paths must remain inside the domain $\Omega$. In Euclidean space, the shortest path between 2 points is the straight line connecting the points. But let's say you live in a world that is shaped like a donut? Then traveling from one side of the donut to the other, while staying inside the donut, will require you to walk in a circular (curved) trajectory.

There exist algorithms which compute exact geodesic distances in meshed domains. See [The Discrete Geodesic Problem](https://www.cs.umd.edu/~mount/Papers/mmp-sicomp-87.pdf), [Fast Exact and Approximate Geodesic Paths on Meshes](https://evasion.inrialpes.fr/Membres/Franck.Hetroy/Teaching/ProjetsImage/2006/Bib/kirsanov_et_al-rr-2004.pdf), for example, for the particular case of surface meshes. In contrast, the heat method aims to compute approximate geodesic distances, but the results can be refined (see [On Variational and PDE-based Distance Function Approximations](https://home.eps.hw.ac.uk/~ab226/papers/pde4dist.pdf), for example).

The general idea behind the heat method is to consider how heat should flow from a source to any other point in the domain $\Omega$ in the most optimal (shortest path) way. Consider a room (i.e. the domain $\Omega$) that is split in half by a wall, but the wall has a hole in its center. Suppose that heat cannot pierce through the walls. Suppose that a heat source is fixed and activated on one side of the wall, and you are on the other side of the wall at the same level. How would heat flow to the other side of the room and reach you? Intuitively, it would go exactly along the wall on the same side as the source, and as soon as it encounters the hole in the wall, it will bend, go through the hole, and bend once more in the same direction, and go along the other side of the wall until it reaches you. If we normalize this heat flow, so that the heat can only ever flow with the same velocity of $1$ meter per second, then we should have essentially reconstructed the gradient of the geodesic distance function, since distance functions must have unit gradient. This leads to the following algorithm, which you will implement (on a mesh of your choosing):

1. Compute a heat flow by first solving the heat equation subject to the initial conditions $u(X,t=0) = 1$ if $X$ is part of the heat source, and 0 otherwise. Solve for this heat for a certain period of time, obtaining $u(X,t)$, the heat at time $t$ (1 time step should be enough). The suggested time step to use (refer to [the paper](https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/)) is a multiple $k$ of the mesh's squared average edge length. Then, compute the negative gradient of the heat, obtaining the heat's flow $-\nabla u(X,t)$.
2. Normalize the flow, obtaining $F(X) = \frac{-\nabla u(X,t)}{|\nabla u(X,t)|}$.
3. Solve for a function $\phi(X)$ whose gradient matches the normalized heat flow, i.e. solve the problem 

    $$
    \min_\phi \int_{\Omega} || \nabla \phi(X) - F(X) ||^2 \partial \Omega .
    $$

    This is a Poisson problem on which FEM can be applied (as shown in the previous section). However, recall that the Laplacian $L$ is rank-deficient (but symmetric negative definite). One can use `pbatoolkit.math.linalg.ldlt` to factorize such a matrix and solve the discrete Poisson equation. However, because the laplacian $\Delta u(X)$ is scale and translation invariant, i.e. $\Delta u(X) = 0$, $\Delta \alpha u(X) = 0$, $\Delta (u(X) + K) = 0$, the Poisson solution may be shifted and/or reflected. Knowing that the distance from the source to the source should be $0$, we can simply shift the whole Poisson solution up by the computed distance at the source (after having handled reflection). If there are many sources, use the average of the computed distances at the sources as the shift.

4. Visualize the heat distribution and the corresponding geodesic distance function on the mesh using `polyscope`. You can also visualize isolines of the distance function using `libigl`.
5. What happens if you add regularization to the Laplacian $L$ in the Poisson solve?

## Least-squares error minimization

An intuitive approach to function approximation is to directly minimize the error between our discrete FEM function $u(X)$ and some target known function $f(X)$ that we wish to approximate. This is mathematically formulated as 

$$
\min_u \frac{1}{2} \int_{\Omega} \left[ u(X) - f(X) \right]^2 \partial \Omega .
$$

If we discretize $u(X)$ using FEM into the coefficient vector $u$ and basis functions $\phi_i(X)$, and we sample the function $f(X)$ at element quadrature points, the resulting minimization becomes

$$
\min_u \frac{1}{2} (N u - f)^T Q (N u - f) .
$$

We can immediately solve this minimization by setting the gradient with respect to our unknown coefficients $u$ to zero. This leads to the linear system of equations

$$
N^T Q N u = N^T Q f \leftrightarrow M u = N^T Q f ,
$$

where $M$ is the mass matrix, and $N^T Q$ is a Galerkin projection operator on the forcing function $f$. This hints at the fact that the forcing function $f$ may also be discretized at the FEM nodes instead, yielding 

$$
Mu = Mf \longrightarrow u = f
$$

which immediately reveals the solution vector $u$ without requiring a linear solver. In this case, we simply need to know how to evaluate the function $f(X)$ at the nodes.

### Signed distance function approximation

Although signed distance functions (SDFs) in general are of great interest in many fields, we restrict our attention to approximate signed distance functions, a popular implicit representation of geometry for efficient contact detection and handling in animation. See [An hp-adaptive discretization algorithm for signed distance field generation](https://ieeexplore.ieee.org/abstract/document/7987773), [Local Optimization for Robust Signed Distance Field Collision](https://dl.acm.org/doi/abs/10.1145/3384538), [Signed distance fields for polygon soup meshes](https://dl.acm.org/doi/10.5555/2619648.2619655), for example.

We will use the framework of FEM function approximation presented in the previous section to approximate SDFs.

1. Construct an FEM (volumetric) mesh of the domain in which you want the SDF to be supported. Use an order 1 mesh in this step. If you want to use a grid-like domain, i.e. using hexahedral FEM elements, make sure that the input hexahedral mesh geometry is given with element vertices ordered according to Lagrange coordinates. This means that the 8 hexahedron vertices should be ordered as `[left-front-bottom, right-front-bottom, left-back-bottom, right-back-bottom, left-front-top, right-front-top, left-back-top, right-back-top]`.
2. Load some surface mesh of your choice. We are going to approximate the signed distance function to that surface.
3. Evaluate the mesh signed distance function to the surface mesh at FEM nodes using `libigl`. Use the winding number variant for robust and accurate distance computation (if your input surface is of poor quality).
4. Refine the volumetric input mesh of the domain and evaluate the FEM discretized approximate SDF at the refined volumetric mesh's vertices. Visualize the SDF on the refined mesh using `polyscope`.
5. Recompute the same SDF FEM approximation pipeline, but now using quadratic shape functions, and then cubic shape functions. Visualize the results on the refined mesh again. Is there a difference?
6. Evaluate the error for all 3 approximations.


### Other applications

Poisson solutions are used in many other works. For example, we can reconstruct an implicit surface from a point cloud using [Poisson Surface Reconstruction](https://hhoppe.com/poissonrecon.pdf) and its numerous derivative works, or enforce incompressibility in fluid simulation for advection-projection schemes (see [Fluid Simulation for Computer Animation](https://www.cs.ubc.ca/~rbridson/fluidsimulation/)), for example.